In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from torch.cuda import amp
from tqdm import tqdm


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [2]:

IMG_SIZE = 224
BATCH_SIZE = 32

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [4]:
DATA_DIR = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake"

train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_tfms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/valid", transform=val_tfms)
test_ds  = datasets.ImageFolder(f"{DATA_DIR}/test", transform=val_tfms)


In [5]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=4, pin_memory=True)

val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=4, pin_memory=True)

test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=4, pin_memory=True)
print("Classes:", train_ds.classes)


Classes: ['fake', 'real']


In [6]:

model = models.vgg16(weights="IMAGENET1K_V1")


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 214MB/s]  


In [7]:
in_features = model.classifier[6].in_features

model.classifier[6] = nn.Linear(in_features, 2)


In [8]:
model = model.to(device)


In [9]:
criterion = nn.CrossEntropyLoss()


In [10]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [11]:
scaler = amp.GradScaler()


/tmp/ipykernel_48/1050304195.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


In [12]:
def train_one_epoch(model, loader, criterion, optimizer, scaler):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Training")

    for imgs, labels in pbar:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

     
        with amp.autocast():
            outputs = model(imgs)
            loss = criterion(outputs, labels)

       
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({
            "loss": total_loss / total,
            "acc": 100 * correct / total
        })

    epoch_loss = total_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc


In [13]:
def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validation")

        for imgs, labels in pbar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            pbar.set_postfix({
                "loss": total_loss / total,
                "acc": 100 * correct / total
            })

    val_loss = total_loss / total
    val_acc = correct / total

    return val_loss, val_acc


In [ ]:
EPOCHS = 3
best_acc = 0

for epoch in range(1, EPOCHS + 1):
    print(f"\nEpoch {epoch}/{EPOCHS}")

    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, scaler
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion
    )

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc*100:.2f}%")


    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_vgg16.pth")
        print("Saved best model → best_vgg16.pth")



Epoch 1/3


Training:   0%|          | 0/3125 [00:00<?, ?it/s]/tmp/ipykernel_48/1878859632.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Training:   2%|▏         | 68/3125 [00:17<12:52,  3.95it/s, loss=1.25, acc=57.6]